In [4]:
#import pertinent packages
import pandas as pd
import math
import numpy as np
import os
import glob

In [5]:
#Insert folder name with csv file that you wish to calculate total error for
FOLDERNAME = os.path.abspath(r'''C:\Users\Marissa\Desktop\Research\Transits\TMMT\TOI-532\20201215_TOI-532\0_results\5_20_30\Best_Reduction''')
regex = "*.xls"
files = glob.glob(os.path.join(FOLDERNAME,regex))
listToStr = ''.join(map(str, files))
df = pd.read_csv(listToStr, sep='\t')

In [6]:
#Enter diameter (D) of telescope in cm (true for both TMMT/LCOBOT)
D = 0.3*100 
#Enter exposure time in  seconds (t)
t = df['EXPTIME']
#Enter airmass of observations (X)
X = df['AIRMASS']
#Enter altitude of telescope in meters (h) (true for both TMMT/LCOBOT)
h = 2286

In [7]:
#Calculate the expected scintillation noise for a given star
sig_s = 0.09 * D ** (-2/3) * X ** (1.75) * (2 * t) ** (-1/2 * (math.e) ** (-h/8000))

In [8]:
#Calculates number of reference stars
ref_stars = [col for col in df.columns if 'rel_flux_C' in col]
n_e = len(ref_stars)
print(n_e)
#Calculates the scintillation error

sig_scint = 1.5 * sig_s * (1 + 1/(n_e)) ** (1/2)

9


In [16]:
#Normalize Rel_Flux_T1 and Rel_Flux_err_T1
med = np.median(df['rel_flux_T1'])
mean = np.mean(df['rel_flux_T1'])
#print(med)
#print(mean)
norm_flux = df['rel_flux_T1'] / med
norm_flux_err = df['rel_flux_err_T1'] / med
print(norm_flux_err)

0      0.003633
1      0.003646
2      0.003633
3      0.003595
4      0.003608
         ...   
165    0.003800
166    0.003838
167    0.003863
168    0.003940
169    0.004043
Name: rel_flux_err_T1, Length: 170, dtype: float64


In [20]:
#Calculates the total error including scintillation

sig_total = ((norm_flux_err) ** 2 + (sig_scint) ** 2) ** (1/2)
print(sig_total)

0      0.006647
1      0.006567
2      0.006479
3      0.006379
4      0.006311
         ...   
165    0.009741
166    0.010008
167    0.010286
168    0.010596
169    0.010926
Length: 170, dtype: float64


In [18]:
#Adds a new column to the end of excel sheet with total error from rel_flux_T1 and scintillation noise added in quadrature
df.insert(len(df.columns), "Total_error", sig_total)
print(df.head)

<bound method NDFrame.head of                                              Label  slice     Saturated  \
0      1  Processed_l20201215-015706-TOI-532-I.fit    0.0  68637.429688   
1      2  Processed_l20201215-015926-TOI-532-I.fit    0.0  68387.898438   
2      3  Processed_l20201215-020144-TOI-532-I.fit    0.0  67780.695312   
3      4  Processed_l20201215-020402-TOI-532-I.fit    0.0  67748.734375   
4      5  Processed_l20201215-020621-TOI-532-I.fit    0.0  68170.812500   
..   ...                                       ...    ...           ...   
165  166  Processed_l20201215-082126-TOI-532-I.fit    0.0      0.000000   
166  167  Processed_l20201215-082345-TOI-532-I.fit    0.0      0.000000   
167  168  Processed_l20201215-082604-TOI-532-I.fit    0.0  63367.082031   
168  169  Processed_l20201215-082822-TOI-532-I.fit    0.0  67136.523438   
169  170  Processed_l20201215-083037-TOI-532-I.fit    0.0  66280.468750   

     J.D.-2400000        JD_UTC       JD_SOBS       HJD_UTC      BJD(

In [19]:
#Creates a new csv file with title editted to include w_tot_error
strip_regex = listToStr.rstrip('.xls')
df.to_csv(strip_regex +'_w_tot_error.xls', sep='\t', index=False)